First, we import stopwords from NLTK for use in our sentiment analysis.

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words("english"))

Next, we ingest our text file of WSB comments to create a list of comments.

In [9]:
comments = []
with open('wsb_comments.txt') as f:
    lines = f.readlines()
    for line in lines:
        comments.append(line)



gonna suck when Tesla beats earnings later this month



Then, we apply a regex filter to our sentences and prune stopwords.

In [11]:
import re
from nltk.stem import PorterStemmer

def clean_comment(comment):
    ps = PorterStemmer()
    regex = re.compile('[^ a-zA-Z]')
    cleaned_comment = regex.sub('', comment)
    tokenized_words = word_tokenize(cleaned_comment.lower())
    cleaned_comments = []

    for word in tokenized_words:
        if word not in stop_words:
            cleaned_comments.append(ps.stem(word))

    return cleaned_comments

In [14]:
processed_comments = []

for comment in comments:
    processed_comments.append(clean_comment(comment))

print(processed_comments)

[['god', 'speed', 'filthi', 'degener'], ['spi', 'put', 'ftw', 'imgemotetth'], ['gon', 'na', 'suck', 'tesla', 'beat', 'earn', 'later', 'month'], ['hey', 'havent', 'alreadi', 'pleas', 'repli', 'comment', 'posit', 'led', 'enter', 'exit'], [], ['bot', 'action', 'perform', 'automat', 'pleas', 'contact', 'moder', 'subredditmessagecomposetorwallstreetbet', 'question', 'concern'], ['mu'], ['dte', 'ban'], ['elon', 'tryna', 'put', 'shit', 'hoooldhttpswwwfactionfinancialcompredictionadedbabfcdcb'], ['tesla', 'put'], ['stop', 'child', 'iq', 'get', 'fuck', 'know', 'damn', 'well', 'imposs'], ['iq', 'intellig', 'quotient', 'smart', 'someon', 'age', 'children', 'averag', 'iq'], ['move', 'tandem', 'short', 'hedgecuckfund', 'one', 'move', 'other', 'move', 'liquid', 'becom', 'issu', 'one', 'move', 'caus', 'move', 'could', 'fault', 'main', 'algorithm', 'use', 'short', 'meme', 'stock', 'break', 'code', 'would', 'appli', 'board'], ['quick', 'googl', 'search', 'yield', 'correl', 'coeffici', 'make', 'sens', '